In [1]:
cd /content/drive/MyDrive/MSDS-20/semester 2/Spring2021 - Deep Learning/Deep learning Assignment 5

/content/drive/MyDrive/MSDS-20/semester 2/Spring2021 - Deep Learning/Deep learning Assignment 5


In [2]:
ls

 Assignment_5.ipynb                'DL_Assignment 5.pdf'   Report.docx
'Assignment 5 Starter Code.ipynb'   encodings.pickle       tokenizer.pickle
'DL A5.ipynb'                       imdb_dataset.csv       unlabeled.csv


In [3]:
pip install pickle5

     |████████████████████████████████| 132 kB 5.2 MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219328 sha256=7c5dfd7d6125042c9f462b9f9bd48197dfdf9eed5a57dbd0e8762f3760b06cd7
  Stored in directory: /root/.cache/pip/wheels/7e/6a/00/67136a90d6aca437d806d1d3cedf98106e840c97a3e5188198
Successfully built pickle5


### Data Preprocessing 
You can use your own way of preprocessing to enhance results. Best results will lead to bonus points.

In [4]:
import pandas as pd
import nltk
import tensorflow as tf
import torch
import torch.nn as nn
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.autograd import Variable
import pickle5 as pickle

In [5]:
reviews = pd.read_csv('imdb_dataset.csv')

In [6]:
reviews.head(10)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


In [7]:
reviews['sentiment'].value_counts()

negative    24000
positive    24000
Name: sentiment, dtype: int64

In [6]:
def preprocess(text):
    lower = text.lower()
    # Removing Punctuation marks
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    rem_punc = tokenizer.tokenize(lower)
    # Removing Stop Words
    stopwords = nltk.corpus.stopwords.words('english')
    rem_stop_words = [word for word in rem_punc if not word in stopwords]
    # Removing Non-English words 
    english_words = nltk.corpus.words.words()
    english_words = [word for word in rem_stop_words if word in english_words]   
    # Insert Start End tokens
    english_words.insert(0,'<start>')
    english_words.append('<end>')
    sentence = ' '.join(english_words)
    return sentence

In [7]:
def encode_text(text):
    # Tokenization
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(text)
    # Converting to sequences
    sequences = tokenizer.texts_to_sequences(text)
    # Padding Zeros 
    tokenizer.word_index['<pad>'] = 0
    tokenizer.index_word[0] = '<pad>'
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, padding='post')
    
    return padded_sequences, tokenizer

In [8]:
# text = list(map(preprocess,reviews.review))
# encodings, tokenizer = encode_text(text)

In [9]:

# with open('encodings.pickle', 'wb') as handle:
#     pickle.dump(encodings, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('tokenizer.pickle', 'wb') as handle:
#     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
with open('encodings.pickle', 'rb') as handle:
    encodings = pickle.load(handle)

In [21]:
X = encodings
y = np.where(np.array(reviews.sentiment)=='positive',1,0)

In [22]:
X.shape,y.shape

((48000, 1053), (48000,))

In [23]:
def makeDataloaders(X,y,batch_size):
    '''
    This is a function that loads the dataset based on the path and sizes for train,test and val sets. 
    It returns dataloaders for training,testing and validation
    '''
    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    # X_train = X_train.astype(float)
    # X_test = X_test.astype(float)
    
    train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
    
    train_loader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

    test_loader=torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)


    return train_loader,test_loader

### Sequence models
Following is an example code for simple LSTMs containing one layer. Your implementation should be generic in which user can be able to create multiple layers if required. 

In [24]:
class LSTM(nn.Module):
    
    def __init__(self,input_dim,n_layers, hidden_dim):

        super().__init__()
        # use pytorch tensors
        # require_grad parameter should be True 
        # Instead of following you can also use nn.Linear()
        self.input_dim  = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 2)

    

    def forward(self, inp):


        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # Initialize hidden state with zeros (batch_size, hidden_dim)
        h0 = torch.zeros(self.n_layers, inp.size(0), self.hidden_dim).float().requires_grad_()
        
        # Initialize cell state with zeros (batch_size, hidden_dim)
        c0 = torch.zeros(self.n_layers, inp.size(0), self.hidden_dim).float().requires_grad_()
        h0, c0 = h0.to(device), c0.to(device)
        # Loop through the whole sequence and update h_t and c_t at every time step
        # Shape of x is (batch_size, dim)
        # out, hidden = self.lstm(inp, (h.detach(), c.detach()))
        out, hidden = self.lstm(inp,(h0,c0))
        # out , hidden  = out.to(device),hidden.to(device)
        # print(hidden.shape)
        out =  self.fc(out.squeeze(1))
        
        return out

In [25]:
def train(model, input_dim,train_loader, val_loader, training_epochs, loss_func, optimizer):
    '''
    This is a function to train the model based on provided parameters
    It outputs train and val accuracy and loss on each epoch and outputs the trained model
    '''
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loss_array,train_acc_array = [], []
    val_loss_array,val_acc_array = [], []
    for epoch in range(training_epochs):
        # Training Phase
        correct = 0
        for batch_idx, (data,target) in enumerate(train_loader):
            # print(batch[0].shape)
            data, target = Variable(data), Variable(target)
            data, target = data.to(device), target.to(device)
            model = model.to(device)
            data = data.view(-1, 1, input_dim).float().requires_grad_()
            optimizer.zero_grad()
            net_out = model(data)
            pred = net_out.data.max(1)[1]  # get the index of the max log-probability
            
            correct += pred.eq(target.data).sum()
            loss = loss_func(net_out, target)
            # train_loss_array.append(loss)
            loss.backward()
            optimizer.step()
            
        train_acc_array.append(correct / len(train_loader.dataset))
        train_loss_array.append(loss.item())
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                epoch, 
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.item(),
                correct, 
                len(train_loader.dataset),
                100. * correct / len(train_loader.dataset)))
                            
        # Validation Phase
        val_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in val_loader:
                data, target = Variable(data), Variable(target)
                data, target = data.to(device), target.to(device)
                data = data.view(-1, 1, input_dim).float().requires_grad_()
                optimizer.zero_grad()
                net_out = model(data)
            
                # sum up batch loss
                val_loss += loss_func(net_out, target).item()
                pred = net_out.data.max(1)[1]  # get the index of the max log-probability
                correct += pred.eq(target.data).sum()

            val_loss /= len(val_loader.dataset)
            val_acc_array.append(correct / len(val_loader.dataset))
            val_loss_array.append(val_loss)
            print('\nVal set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                val_loss, 
                correct, 
                len(val_loader.dataset),
                100. * correct / len(val_loader.dataset)))
    
    return model, train_loss_array,train_acc_array,val_loss_array,val_acc_array


In [46]:
input_dim = 1053
training_epochs = 20
hidden_dim = 150
n_layers = 20
model = LSTM(input_dim = input_dim,hidden_dim = hidden_dim,n_layers = n_layers)
loss = nn.CrossEntropyLoss()
lr = 0.02
optimizer = torch.optim.SGD(model.parameters(), lr=lr)  
train_loader,test_loader = makeDataloaders(X,y,batch_size=32)
model, train_loss_array,train_acc_array,val_loss_array,val_acc_array = train(model,input_dim,train_loader, test_loader,
                                                                            training_epochs = training_epochs, 
                                                                            loss_func = loss, optimizer = optimizer)


Train Epoch: 0 [38368/38400 (100%)]	Loss: 0.689466, Accuracy: 19080/38400 (50%)


Val set: Average loss: 0.0217, Accuracy: 4800/9600 (50%)

Train Epoch: 1 [38368/38400 (100%)]	Loss: 0.691795, Accuracy: 19046/38400 (50%)


Val set: Average loss: 0.0217, Accuracy: 4800/9600 (50%)

Train Epoch: 2 [38368/38400 (100%)]	Loss: 0.693435, Accuracy: 19254/38400 (50%)


Val set: Average loss: 0.0217, Accuracy: 4800/9600 (50%)

Train Epoch: 3 [38368/38400 (100%)]	Loss: 0.693591, Accuracy: 19122/38400 (50%)


Val set: Average loss: 0.0217, Accuracy: 4800/9600 (50%)

Train Epoch: 4 [38368/38400 (100%)]	Loss: 0.693280, Accuracy: 19220/38400 (50%)


Val set: Average loss: 0.0217, Accuracy: 4800/9600 (50%)

Train Epoch: 5 [38368/38400 (100%)]	Loss: 0.693129, Accuracy: 19318/38400 (50%)


Val set: Average loss: 0.0217, Accuracy: 4800/9600 (50%)

Train Epoch: 6 [38368/38400 (100%)]	Loss: 0.693258, Accuracy: 19074/38400 (50%)


Val set: Average loss: 0.0217, Accuracy: 4800/9600 (50%)

Train Epoch: 7 [3836

In [ ]:
input_dim = 1053
training_epochs = 5
hidden_dim = 150
n_layers = 20
model = LSTM(input_dim = input_dim,hidden_dim = hidden_dim,n_layers = n_layers)
loss = nn.CrossEntropyLoss()
lr = 0.02
optimizer = torch.optim.SGD(model.parameters(), lr=lr)  
train_loader,test_loader = makeDataloaders(X,y,batch_size=32)
model, train_loss_array,train_acc_array,val_loss_array,val_acc_array = train(model,input_dim,train_loader, test_loader,
                                                                            training_epochs = training_epochs, 
                                                                            loss_func = loss, optimizer = optimizer)


print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {dev_loss:.3f} | Val. Acc: {dev_acc*100:.2f}% |')

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 01 | Train Loss: 0.691 | Train Acc: 52.55% | Val. Loss: 0.687 | Val. Acc: 56.56% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 02 | Train Loss: 0.656 | Train Acc: 61.22% | Val. Loss: 0.622 | Val. Acc: 64.79% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06
0% [██████████████████████████████] 100% | ETA: 00:00:00

...


Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 04 | Train Loss: 0.626 | Train Acc: 65.03% | Val. Loss: 0.600 | Val. Acc: 67.38% |



Total time elapsed: 00:00:00
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06
0% [██████████████████████████████] 100% | ETA: 00:00:00

| Epoch: 05 | Train Loss: 0.616 | Train Acc: 65.61% | Val. Loss: 0.595 | Val. Acc: 68.09% |



Total time elapsed: 00:00:00


##GRU

### Word Cloud

In [32]:
# Word Cloud that shows the positive and negative words and thier frequence 
# make a dictionary of words with their word_count
counts = tokenizer.word_counts
# use you trained model to prodict sentiment of these words ??
# Saperate positive sentiment words and negative sentiment words with their counts ??
# make word cloud according
plt.figure(figsize=(??,??))
wordcloud = WordCloud(background_color=??,max_words=??).generate_from_frequencies(??)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

SyntaxError: ignored

In [ ]:
# Word Cloud that shows main words used in reviews that are positive or negative  
# Randomly select one sentence that has positive sentiment and one with negative sentiment 
??
# make word cloud of those sentences and see what are main words used in positive review and in negative reviews
# Similarly you can make word clouds of all positive reviews and negative reviews to see the most repeating words in positive sentiments and negative sentiments
plt.figure(figsize=(??,??))
plt.title('Prediceted Sentiment on this review: '+??)
wordcloud = WordCloud(background_color='White',max_words=100).generate(??)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()